# Анализ графов в Python
Тетрадка посвящена работе с графами в питоне, а так же приведён пример анализа сети Facebook.

## Обзор NetworkX

NetworkX --- это питонячая библиотека, предназначенная для создания, обработки и изучения сложных сетей (aka графов). В отличие от других библиотек, которые вы можете встретить в сети (igraph, graphviz etc), она полностью написана на Python, благодаря чему ставится через Anaconda и не требует предустановки других пакетов.

Материал сильно опирается на лекцию по [введению в NetworkX](https://www.cl.cam.ac.uk/~cm542/teaching/2010/stna-pdfs/stna-lecture8.pdf).

In [ ]:
import networkx as nx

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Пример создания простого графа:

In [ ]:
# создаём заготовку графа, в которую будут добавляться вершины и рёбра
g = nx.Graph()

Добавление вершин:

In [ ]:
# вершину графа можно называть как строкой, так и числом.
g.add_node(1)
g.add_node('n')

# добавление нескольких вершин сразу
g.add_nodes_from([2, 3])

Добавление рёбер:

In [ ]:
# указываем две вершины, которые надо соединить
g.add_edge(1, 'n')

# добавление нескольких рёбер между указанными вершинами
g.add_edges_from([(1, 2), (1, 3)])

# добавление двух вершин с ребром между ними
# создаются и новые вершины, и ребро между ними
g.add_edge('a', 'b', weight=0.1)
g.add_edge('b', 'c', weight=1.5)
g.add_edge('a', 'c', weight=1)
g.add_edge('c', 'd', weight=0.2)
g.add_edge('c', 153, weight=1.2)

Визуализация (при повторном запуске ячейки картинка может меняться):

In [ ]:
nx.draw_networkx(g, node_color='lightblue')
plt.axis('off')

print('Graph nodes:', g.nodes())

Вывод кратчайшего (или минимизирующего затраты) пути от одной вершины до другой:

In [ ]:
# без учёта весов
print(nx.shortest_path(g, 'b', 'd'))

# с учётом весов: теперь за переход по каждому ребру вносится плата
print(nx.shortest_path(g, 'b', 'd', weight='weight'))

Точно так же, как ребру приписывался численный параметр `weight`, вершине можно приписывать любые характеристики:

In [ ]:
g.add_node('node',
           date='06.05.2018',
           any_name='some information')
g.node['node']

Т.е. каждый узел графа можно воспринимать как питонячий словарь `dict` с произвольными ключами и значениями.

Фактически весь граф это просто словарь, где ключ -- номер вершины, значение -- набор вершин, соседних к ней:

In [ ]:
g.adj

In [ ]:
g['c']

Можно вывести и другое традиционное представление графа в виде матрицы смежности:

In [ ]:
nx.adjacency_matrix(g).todense()

Получение числа вершин графа:

In [ ]:
g.number_of_nodes()

In [ ]:
len(g)

Числа рёбер:

In [ ]:
g.number_of_edges()

Совокупная информация:

In [ ]:
print(nx.info(g))

При работе с графами часто бывает полезным получить доступ ко всем вершинам / рёбрам, что позволяет сделать метод `.nodes()` / `.edges()`:

In [ ]:
g.nodes(data=True)

In [ ]:
g.edges(data=True)

In [ ]:
for node in g.nodes():
    print(node, g.degree(node))

---

Закодим простой граф.

Создайте граф, содержащий $20$ вершин с именами $0, 1, \ldots, 19$:

In [ ]:
new_g = #YOUR CODE
#YOUR CODE

Добавьте рёбра, соединяющие вершины через одну ($0$ и $2$, $1$ и $3$, ..., $18$ и $0$, $19$ и $1$):

In [ ]:
#YOUR CODE

Нарисуйте полученный граф:

In [ ]:
#YOUR CODE

## Почти реальные данные

Считаем данные о пьесах Шекспира:

In [ ]:
shakespeare_plays = pd.read_csv('Shakespeare_data.csv')

shakespeare_plays.head()

Замените пропущенные значения в столбце `Player` на `Other`:

In [ ]:
#YOUR CODE

Выведем названия пьес:

In [ ]:
print('\n'.join(shakespeare_plays['Play'].unique()))

Общее число пьес и действующих лиц:

In [ ]:
len(shakespeare_plays['Play'].unique()), len(shakespeare_plays['Player'].unique())

Составим все пары пьес, обладающих общими действующими лицами:

In [ ]:
play_to_player = shakespeare_plays[['Play', 'Player']].drop_duplicates()

play_to_play = pd.merge(play_to_player, play_to_player, on='Player', how='inner')

play_to_play.head()

Загрузим данные в граф из подготовленной таблицы:

- вершинами будут названия пьес
- ребро между двумя пьесами проводится, если их списки действующих лиц пересекаются

In [ ]:
plays = nx.from_pandas_edgelist(play_to_play[['Play_x', 'Play_y']].drop_duplicates(),
                                source='Play_x', target='Play_y')

print(nx.info(plays))

Нарисуем граф и сохраним картинку в высоком разрешении, чтобы её можно было открыть и нормально рассмотреть:

In [ ]:
plt.figure(figsize=(40, 40)) 

nx.draw(plays, with_labels=True, node_color='white', node_size=4000)

plt.savefig('shakespear_plays.png', bbox_inches='tight')

Выведем все степени вершин:

In [ ]:
plays.degree

Постройте гистограмму степеней вершин:

In [ ]:
#YOUR CODE

plt.xlabel('Degree')
plt.ylabel('Number of plays')
plt.title('Shakespear\'s plays connectivity degrees')

По полученным результатам делаем вывод, что набор персонажей пьес достаточно типичен.

Выведите пьесу, которая наиболее сильно пересекается по персонажам с другими (т.е. вершина пьесы имеет наибольшую степень):

In [ ]:
#YOUR CODE

Выведите число персонажей для полученной пьесы.

*Спойлер: графы тут ни при чём, pandas в помощь.*

In [ ]:
#YOUR CODE

Посмотрим на персонажей полученной пьесы, которые присутствуют в других пьесах:

In [ ]:
play_name = #YOUR CODE
play_to_play[play_to_play.Play_x == play_name]['Player'].unique()

Библиотека NetworkX позволяет найти кратчайший путь между вершинами, по которому можно судить об их взаимном расположении:

In [ ]:
nx.shortest_path(plays, 'macbeth', 'The Tempest')

In [ ]:
nx.shortest_path(plays, 'macbeth', 'Timon of Athens')

А ещё можно вывести кратчайшие пути до всех вершин от данной:

In [ ]:
nx.single_source_shortest_path(plays, 'macbeth')

Выделим некоторые кратчайшие пути цветом (при желании можно сохранить картинку, как и раньше).

In [ ]:
# функция для выделения цветом путей, пример использования ниже
# https://github.com/jtorrents/pydata_bcn_NetworkX/blob/master/NetworkX_SNA_workshop_with_solutions.ipynb
def plot_paths(G, paths):
    plt.figure(figsize=(36, 36))
    pos = nx.fruchterman_reingold_layout(G)
    nx.draw_networkx_nodes(G, pos=pos, node_size=4000, node_color='white')
    nx.draw_networkx_labels(G, pos=pos, labels={n: n for n in G})
    # Draw edges
    nx.draw_networkx_edges(G, pos=pos)
    for path in paths:
        edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos=pos, edgelist=edges, edge_color='red', width=3)
    ax = plt.gca()
    ax.set_axis_off()
    ax.grid(None)

In [ ]:
# вывод всех кратчайших путей из одной
plot_paths(plays, nx.single_source_shortest_path(plays, 'macbeth').values())

In [ ]:
# вывод нескольких кратчайших путей
plot_paths(plays, [nx.shortest_path(plays, 'macbeth', 'The Tempest'), nx.shortest_path(plays, 'Hamlet', 'Romeo and Juliet')])

### Метрики на графе

In [ ]:
degree = nx.degree_centrality(plays)
betweenness = nx.betweenness_centrality(plays)
closeness = nx.closeness_centrality(plays)
eigen = nx.eigenvector_centrality(plays)

In [ ]:
graph_measures = {
    'degree': degree,
    'betweenness': betweenness,
    'closeness': closeness,
    'eigenvector': eigen,
}

pd.DataFrame(graph_measures)

Посмотрим, какие пьесы обладают максимальными показателями и проинтерпретируем:

In [ ]:
pd.DataFrame(graph_measures).sort_values(by='eigenvector', ascending=False)

В данном случае метрики достаточно сильно коррелируют между собой, в реальной жизни часто бывает не так:

In [ ]:
import seaborn as sns

sns.pairplot(pd.DataFrame(graph_measures))

**Bonus task**

Ниже создаётся следующий граф:
- вершинами являются имена действующих лиц
- два персонажа соединяются, если появлялись одновременно хотя бы в одной пьесе

In [ ]:
player_to_player = pd.merge(play_to_player, play_to_player, on='Play', how='inner')

player_to_player.head()

In [ ]:
players = nx.from_pandas_edgelist(player_to_player[['Player_x', 'Player_y']].drop_duplicates(),
                                  source='Player_x', target='Player_y')

print(nx.info(players))

In [ ]:
plt.figure(figsize=(40, 40)) 

nx.draw(players, with_labels=True, node_color='white', node_size=4000)

plt.savefig('shakespear_players.png', bbox_inches='tight')

Проведите анализ, аналогичный проведённому по пьесам.

Какой персонаж "знаком" с большинством других? Что можно сказать по метрикам на полученном графе?

## Реальные данные

Подгрузка данных из файла-не таблицы осуществляется достаточно просто, если данные записаны в правильном формате. 

Примеры наиболее популярных форматов для чтения и сохранения графов (больше можно найти в документации NetworkX):
- список смежных вершин (`nx.read_adjlist`, `nx.write_adjlist`, именно так хранятся графы в NetworkX)
- список всех рёбер (`nx.read_edgelist`, `nx.write_edgelist`)

Первые строки нашего файла `facebook_combined.txt` выглядят так:
```
214328887 34428380
17116707 28465635
380580781 18996905
221036078 153460275
107830991 17868918
151338729 222261763
```

Каждое число обозначает имя вершины (грубо говоря, id пользователя) в графе. Если в одной строке записана пара чисел, значит, пользователи с соответствующими номерами находятся друг у друга в списке друзей.

In [ ]:
facebook_users = nx.read_edgelist("facebook_combined.txt")

Посмотрим, сколько граф содержит вершин и связей:

In [ ]:
print('Number of nodes:', facebook_users.number_of_nodes())
print('Number of edges:', facebook_users.number_of_edges())

Попробуем нарисовать граф (может занять около минуты, поскольку вершин достаточно много):

In [ ]:
%%time
plt.figure(figsize=(16, 16))
nx.draw_networkx(facebook_users, node_color='lightblue', with_labels=False, node_size=50, alpha=0.5)
plt.axis('off')

In [ ]:
degrees = dict(facebook_users.degree()) # dictionary node:degree
values = sorted(set(degrees.values()))
g_hist = [list(degrees.values()).count(x) for x in values]

plt.figure(figsize=(7, 5))
plt.plot(values, g_hist, 'o-') # degree

plt.xlabel('Degree')
plt.ylabel('Number of nodes')
plt.title('Facebook users connectivity degrees')

Это считается быстро:

In [ ]:
%%time
degree = nx.degree_centrality(facebook_users)

In [ ]:
%%time
eigen = nx.eigenvector_centrality(facebook_users)

А это -- несколько минут, можно запустить заранее:

In [ ]:
%%time
betweenness = nx.betweenness_centrality(facebook_users)

In [ ]:
%%time
closeness = nx.closeness_centrality(facebook_users)

In [ ]:
graph_measures = {
    'degree': degree,
    'betweenness': betweenness,
    'closeness': closeness,
    'eigenvector': eigen,
}

pd.DataFrame(graph_measures).head()

In [ ]:
import seaborn as sns

sns.pairplot(pd.DataFrame(graph_measures))

## Обзор библиотеки igraph

Помимо NetworkX можно пользоваться и другими библиотеками, например, igraph.

Она реализована на С и быстрее работает с большими графами.

In [ ]:
import igraph

Создание графа практически не отличается от создания в NetworkX, только при добавлении вершин в данном случае указывается их количество, а не имена:

In [ ]:
g = igraph.Graph()

g.add_vertices(3)
g.add_edges([(0, 1), (1, 2), (0, 2)])

print(g)

Пояснение вывода:
```
IGRAPH U--- <число вершин> <число рёбер> --
+ edges:
<перечисление рёбер в формате <вершина>--<вершина>>
```

Доступ к вершинам и рёбрам осуществляется по индексу:

In [ ]:
g.vs[0]

In [ ]:
g.es[0]

Подсчёт степени для всех вершин и для конкретной:

In [ ]:
g.degree()

In [ ]:
g.degree(1)

Загрузим данные о пользователях Facebook:

In [ ]:
facebook_graph = igraph.Graph()

In [ ]:
facebook_graph = facebook_graph.Read_Edgelist("facebook_combined.txt", directed=False)

In [ ]:
print(facebook_graph)

Посчитаем метрики на графе (сравните скорость работы!):

In [ ]:
%%time
degree = facebook_graph.degree()

In [ ]:
%%time
eigen = facebook_graph.eigenvector_centrality()

In [ ]:
%%time
betweenness = facebook_graph.betweenness()

In [ ]:
%%time
closeness = facebook_graph.closeness()

In [ ]:
graph_measures = {
    'degree': degree,
    'betweenness': betweenness,
    'closeness': closeness,
    'eigenvector': eigen,
}

pd.DataFrame(graph_measures).head()

In [ ]:
import seaborn as sns

sns.pairplot(pd.DataFrame(graph_measures))